In [1]:
from future.builtins import next
import re
from numpy import nan
import dedupe
from unidecode import unidecode
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.session import SparkSession
import json
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType,FloatType
import warnings
warnings.simplefilter(action='ignore')

In [2]:
spark = SparkSession \
    .builder \
    .appName("dedupe") \
    .master("local[*]") \
    .getOrCreate()

22/10/31 12:10:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [17]:
# input_file = '/home/abhishekks446/data/netflix_titles.csv'


primart_key="InvoiceNo"


df1= spark.read.option("header","true").csv("/home/abhishekks446/Downloads/inventory-4k.csv")
df2= spark.read.option("header","true").csv("/home/abhishekks446/Downloads/inventory-4k.csv")


fields = [
{'field' : 'Country', 'type': 'String'},
{'field' : 'Description', 'type': 'String'},
{'field' : 'Quantity', 'type': 'String'}
]

In [18]:
def preProcess(column):
    """
    Do a little bit of data cleaning with the help of Unidecode and Regex.
    Things like casing, extra spaces, quotes and new lines can be ignored.
    """
    import unidecode
    # column = column.decode("utf8")
    column = unidecode.unidecode(column)
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    if not column :
        column = None
    return column
def readData(df):
    """
    Read in our data from a CSV file and create a dictionary of records,
    where the key is a unique record ID and each value is dict
    """
    
    data_d = {}
    dfn=df.na.fill("null")
    results = dfn.toJSON().map(lambda j: json.loads(j)).collect()
    for row in results:
        clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
        row_id = row[primart_key]
        data_d[row_id] = dict(clean_row)

    return data_d
data_d = readData(df1)
data_s = readData(df2)

In [19]:
print(data_d)

{'INV-1': {'InvoiceNo': 'inv-1', 'StockCode': '85123a', 'Description': 'white hanging heart t-light holder', 'Quantity': '6', 'InvoiceDate': '12/1/2010 8:26', 'UnitPrice': '2.55', 'CustomerID': '17850', 'Country': 'united kingdom'}, 'INV-2': {'InvoiceNo': 'inv-2', 'StockCode': '71053', 'Description': 'white metal lantern', 'Quantity': '6', 'InvoiceDate': '12/1/2010 8:26', 'UnitPrice': '3.39', 'CustomerID': '17850', 'Country': 'united kingdom'}, 'INV-3': {'InvoiceNo': 'inv-3', 'StockCode': '84406b', 'Description': 'cream cupid hearts coat hanger', 'Quantity': '8', 'InvoiceDate': '12/1/2010 8:26', 'UnitPrice': '2.75', 'CustomerID': '17850', 'Country': 'united kingdom'}, 'INV-4': {'InvoiceNo': 'inv-4', 'StockCode': '84029g', 'Description': 'knitted union flag hot water bottle', 'Quantity': '6', 'InvoiceDate': '12/1/2010 8:26', 'UnitPrice': '3.39', 'CustomerID': '17850', 'Country': 'united kingdom'}, 'INV-5': {'InvoiceNo': 'inv-5', 'StockCode': '84029e', 'Description': 'red woolly hottie w

In [6]:
# df1.printSchema()

In [20]:
deduper = dedupe.RecordLink(fields)
# deduper = dedupe.Gazetteer(fields)

In [21]:

deduper.prepare_training(data_d,data_s)

In [22]:
dedupe.console_label(deduper)

Country : united kingdom
Description : metal sign,cupcake single hook
Quantity : 1

Country : united kingdom
Description : metal sign cupcake single hook
Quantity : 1

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


y


Country : united kingdom
Description : metal sign,cupcake single hook
Quantity : 1

Country : united kingdom
Description : metal sign cupcake single hook
Quantity : 1

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


y


Country : united kingdom
Description : fairy cakes notebook a6 size
Quantity : 1

Country : united kingdom
Description : fairy cakes notebook a7 size
Quantity : 1

2/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


...

Country : united kingdom
Description : 72 sweetheart fairy cake cases
Quantity : 24

Country : united kingdom
Description : 72 sweetheart fairy cake cases
Quantity : 1

9/10 positive, 12/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


y


Country : united kingdom
Description : clear drawer knob acrylic edwardian
Quantity : 12

Country : united kingdom
Description : clear drawer knob acrylic edwardian
Quantity : 10

10/10 positive, 12/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


f


Finished labeling


In [8]:
# with open('/home/abhishekks446/data/output_num/label.json') as f:
#     deduper.prepare_training(data_d, data_s, training_file=f)

In [23]:
deduper.train()
with open("./4klabel.json", 'w') as tf:
            deduper.write_training(tf)

In [24]:
dupes_outout = deduper.join(
                    data_d, data_s, threshold=0.5)

In [25]:
lis = [list(elem) for elem in dupes_outout]
ls = []
for row in lis:
    a = list(row[0])
    lst = [str(a[0]), str(a[1]), float(row[1])]
    ls.append(lst)
columns = ["df1", "df2", "confidence_score"]
result_data = spark.createDataFrame(ls, columns)


In [26]:
result_data.show()

+----------+----------+------------------+
|       df1|       df2|  confidence_score|
+----------+----------+------------------+
| INVOICE-9| INVOICE-9|0.8856862187385559|
| INVOICE-8| INVOICE-8|0.8856862187385559|
| INVOICE-7| INVOICE-7|0.8856862187385559|
| INVOICE-6| INVOICE-6|0.8856862187385559|
| INVOICE-5| INVOICE-5|0.8856862187385559|
| INVOICE-4| INVOICE-4|0.8856862187385559|
| INVOICE-3| INVOICE-3|0.8856862187385559|
| INVOICE-2| INVOICE-2|0.8856862187385559|
|INVOICE-11|INVOICE-11|0.8856862187385559|
|INVOICE-10|INVOICE-10|0.8856862187385559|
| INVOICE-1| INVOICE-1|0.8856862187385559|
|   INV-999|   INV-999|0.8856862187385559|
|   INV-998|   INV-998|0.8856862187385559|
|   INV-997|   INV-997|0.8856862187385559|
|   INV-996|   INV-996|0.8856862187385559|
|   INV-995|   INV-995|0.8856862187385559|
|   INV-994|   INV-994|0.8856862187385559|
|   INV-993|   INV-993|0.8856862187385559|
|   INV-992|   INV-992|0.8856862187385559|
|   INV-991|   INV-991|0.8856862187385559|
+----------

In [30]:
dfi= spark.read.option("header","true").csv("/home/abhishekks446/data/demo_20_indexed.csv")
def preProcess(column):
    """
    Do a little bit of data cleaning with the help of Unidecode and Regex.
    Things like casing, extra spaces, quotes and new lines can be ignored.
    """
    import unidecode
    # column = column.decode("utf8")
    column = unidecode.unidecode(column)
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    if not column :
        column = None
    return column
def readData(df):
    """
    Read in our data from a CSV file and create a dictionary of records,
    where the key is a unique record ID and each value is dict
    """
    
    data_d = {}
    dfn=df.na.fill("null")
    results = dfn.toJSON().map(lambda j: json.loads(j)).collect()
    for row in results:
        clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
        row_id = row["indexed_records"]
        data_d[row_id] = dict(clean_row)

    return data_d
data_i = readData(dfi)

In [31]:
deduper.search(data_i, threshold=0.5, n_matches=2)

OperationalError: no such table: indexed_records

In [ ]:
# print('clustering...')
# clustered_dupes = deduper.partition(data_d, .5)

# print('# duplicate sets', len(clustered_dupes))

In [16]:
clustered_dupes=deduper.join(data_d,data_s, threshold=0.5,constraint='one-to-one')
# deduper.search(data_d,data_s, threshold=0.2)

In [17]:
clustered_dupes

[(['df10', 'dd9'], 0.99891025),
 (['df11', 'dd11'], 0.99891025),
 (['df12', 'dd12'], 0.99891025),
 (['df13', 'dd13'], 0.99891025),
 (['df14', 'dd14'], 0.99891025),
 (['df14', 'dd15'], 0.9854765),
 (['df15', 'dd14'], 0.9854765),
 (['df15', 'dd15'], 0.99891025),
 (['df16', 'dd16'], 0.99891025),
 (['df17', 'dd17'], 0.99891025),
 (['df18', 'dd17'], 0.996866),
 (['df19', 'dd19'], 0.99891025),
 (['df20', 'dd20'], 0.99891025),
 (['df6', 'dd6'], 0.99891025),
 (['df6', 'dd7'], 0.70774895),
 (['df7', 'dd7'], 0.99891025),
 (['df7', 'dd6'], 0.70774895),
 (['df8', 'dd8'], 0.99891025),
 (['df9', 'dd9'], 0.99891025)]

In [ ]:
lis=[list(elem) for elem in clustered_dupes]


In [ ]:
ls=[]
for row in lis:
    a=list(row[0])
    c=[str(a[0]),str(a[1]),float(row[1])]
    ls.append(c)
    

In [ ]:
print(ls)

In [ ]:
columns = ["df1", "df2","confidence_score"]    
dataframe = spark.createDataFrame(ls,columns)

In [ ]:
dataframe.show()